In [2]:
# create datacard for 2 tag

import ROOT as rt
import csv
import re
import sys
import collections
import os

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
sys.path.append('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/python/')
from helper import make_datacard_2sig, make_datacard_2tag, weight_calc
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import deltaR, deltaPhi

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0


print(sys.version)

Welcome to JupyROOT 6.18/02
3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [7]:
fpath =OrderedDict()
tree = OrderedDict()
mass = [15, 40, 55]


old_ctau = {
#     'STodd_ms3p0':[100,500],
#     'SToEE_ms0p4':[10,50],
# #     'SToEE_ms1p0':[25,125],
# #     'SToEE_ms2p0':[50,250],
# #     'SToEE_ms4p0':[100,500],
#     'SToGammaGamma_ms0p4':[10,50],
# #     'SToGammaGamma_ms1p0':[25,125],
# #     'SToGammaGamma_ms2p0':[50,250],
# #     'SToGammaGamma_ms4p0':[100,500],
# #     'SToGlueGlue_ms3p0':[100,500],
#     'SToKPlusKMinus_ms1p5':[38,187],
#     'SToK0K0_ms1p5':[38,187],
#     'SToPi0Pi0_ms0p4':[10,50],
#     'SToPi0Pi0_ms1p0':[25,125],
#     'SToPiPlusPiMinus_ms0p4':[10,50],
#     'SToPiPlusPiMinus_ms1p0':[25,125],
#     'SToPiPlusPiMinus_ms2p0':[50,250],
#     'SToPiPlusPiMinus_ms4p0':[100,500],
    'STodd_ms7':[100, 1000, 10000, 100000],
    'STodd_ms15':[100, 1000, 10000, 100000],
    'STodd_ms40':[100, 1000, 10000, 100000],
    'STodd_ms55':[100, 1000, 10000, 100000],
    'SToTauTau_ms7':[100, 1000, 10000, 100000],
    'SToTauTau_ms15':[100, 1000, 10000, 100000],
    'SToTauTau_ms40':[100, 1000, 10000, 100000],
    'SToTauTau_ms55':[100, 1000, 10000, 100000],
    'SToBB_ms15':[100, 1000, 10000, 100000],
    'SToBB_ms40':[100, 1000, 10000, 100000],
    'SToBB_ms55':[100, 1000, 10000, 100000],
    
}



prod = ['ggH', 'VBFH','ZH', 'WH', 'ttH','ggZH']
prod = ['ggH']

ntupler_version = 'V1p17/'
mc_path = {}
analyzer_version = 'v1/v181/'
mc_central_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
analyzer_version = 'v2/v181/'
mc_private_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'


for p in prod:
#     if 'WH' in p:continue
    for k in old_ctau.keys():
        for ct in old_ctau[k]:

            key = 'MC_'+p+'_'+k + '_'+str(ct)
            if (p == 'ggH' or p == 'VBFH') and ('ms7' in k or 'ms15' in k or 'ms40' in k or 'ms55' in k) and not (p =='VBFH' and 'dd' in k):
                mass = k[k.find('ms')+2:]
                if 'dd' in k and p == 'ggH':fpath[key] = mc_central_path+p+'_HToSSTodddd_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
                elif 'BB' in k and p =='ggH':fpath[key] = mc_central_path+p+'_HToSSTobbbb_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
                elif 'BB' in k and p =='VBFH':fpath[key] = mc_central_path+p+'_HToSSTo4b_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
                else:fpath[key] = mc_central_path+p+'_HToSSTo4Tau_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
            else:
                
                if 'ggH' in p or 'ttH' in p: fpath[key] = mc_private_path+p+'_HToSS_'+k+'_pl'+str(ct)+'_137000pb_weighted.root'
                elif p == 'VBFH' or p=='ZH' or p == 'WH': fpath[key] = mc_private_path+p+'ToSS_'+k+'_pl'+str(ct)+'_137000pb_weighted.root'
                elif 'ggZH' in p:fpath[key] = mc_private_path+'ZHToSS_'+k+'_pl'+str(ct)+'_137000pb_weighted.root'
            if not os.path.exists(fpath[key]):print(key, fpath[key])
            
NEvents = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
#     NEvents[k] = root_dir['NEvents'][1]
    NEvents[k] = root_dir['NEvents']._fEntries
    
#     if not 'data' in k: 
#         print(k, root_dir['NEvents']._fEntries)


root_dir = uproot.open('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ZHToggZH_HiggsPtReweight.root') 
h_reweight = root_dir['higgsPthiggsEta']

# nCsc with different hit vetoing

In [9]:

jetPt_cut = 50

weight = {}
nhits1 = {}
nhits2 = {}
sel_ev = {}
cond = {}
ggZH_weight = {}
higgsEta = {}
higgsPt = {}
cluster_index = ''
addNoiseFlag = 1
# 0: 2 CSC; 1: 2DT; 2: csc+dt
for category in [0,1,2]:
    
    for key in list(tree.keys()):
        if 'data' in key:continue
    ########### SELECTION: CLUSTERS ############
        if 'data' in key: T = tree['data']
        else: T = tree[key]
        k = key + '_'+str(category)
        sel_csccluster = T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll')<20
        sel_csccluster = np.logical_and(sel_csccluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))<1.2)
        sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt')<30)
        sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio')<1)
#         sel_csccluster = np.logical_and(sel_csccluster, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 30, T.array('cscRechitClusterMuonVetoGlobal'))))
        sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitClusterGlobalMuonVetoPt') < 30)

        if 'data' in k: 
            sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')< 12.5, \
                                                                             T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5))
        else: 
            sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')+0.66 < 12.5, \
                                                                             T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')+0.66 > -5))

        sel_dtcluster = np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1
#         sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))

        sel_dtcluster = np.logical_and(sel_dtcluster, T.array('dtRechitClusterLooseIdMuonVetoPt') < 10)

        sel_dtcluster = np.logical_and(sel_dtcluster, np.abs(T.array('dtRechitClusterJetVetoPt')) < 50)
        sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))


        cut = 5
        station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
    +(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)

        max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
        min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))

        sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations
        if addNoiseFlag and not 'data' in k: 
            sel_dtcluster = np.logical_and(sel_dtcluster, (T.array('dtRechitClusterSize')+T.array('dtRechitClusterNoiseHit')) >= 50)
        else: sel_dtcluster = np.logical_and(sel_dtcluster, T.array('dtRechitClusterSize') >= 50)




        ###################### cosmic muon veto #############
        sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
        sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
        sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
        sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                                   T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
        nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
        +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)

        sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))

    ########### SELECTION: JETS ############

        sel_jet = np.logical_and(T.array('jetPt') > 30, np.abs(T.array('jetEta')) < 2.4 )
        sel_jet = np.logical_and(T.array('jetTightPassId'), sel_jet)



    ########### SELECTION: NOISE IN DT ############

        spike = np.logical_and( T.array('nDTRechitsSector')[:,0,0,7]>50,  T.array('nDTRechitsSector')[:,0,0,7]+T.array('nDTRechitsSector')[:,0,0,8]+T.array('nDTRechitsSector')[:,0,0,9]>120)
        spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,8]>25)
        spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,9]>10)



    ########### SELECTION: EVENTS ############

        sel_ev[k] = T.array('METNoMuTrigger')
        sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise') >= 200)
        sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
        sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
        sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))
        sel_ev[k] = np.logical_and(sel_ev[k] , np.logical_not(spike))

    ########### BRANCHES ############




        if category == 0:
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 2)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 0)
            cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                            T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,1], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,1])<2


            nhits1[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,0]
            nhits2[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,1]
    #         deltaRCluster[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
    #                         T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,1], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,1])
        elif category == 1:
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 2)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 0)
            cond[k] = np.abs(T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0]-\
                                                        T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,1])>=0
            if addNoiseFlag and not 'data' in k:
                nhits1[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][:,0]
                nhits2[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][:,1]
            else:
                nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,0]
                nhits2[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,1]
        elif category == 2:
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 1)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum() == 1)

    #         cond[k] =  np.abs(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0]-\
    #                                                     T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0])<2.0

            cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                            T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0], T.array('dtRechitCluster' + cluster_index + 'Phi')[sel_dtcluster][sel_ev[k]][:,0])<2.5
            if addNoiseFlag and not 'data' in k:
                nhits1[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
            else:
                nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
            nhits2[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,0]



        else:
            assert(False)


        higgsPt[k] = T.array('gHiggsPt')[sel_ev[k]][cond[k]]
        higgsEta[k] = T.array('gHiggsEta')[sel_ev[k]][cond[k]]

        ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]

    #     if 'ggH' in k: weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev[k]]


        if 'ggH' in k: weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev[k]][cond[k]]
        else:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev[k]][cond[k]]
        if 'ggZH' in k: weight[k] *= ggZH_weight[k]


In [10]:
BR = 0.01

tables = ['0p4', '1p0', '1p5', 'dd', 'TauTau', 'BB']
ctau_list = {
    '0p4': [10,50],
    '1p0': [25, 125],
    '1p5': [38, 187],
    'dd': [100, 500, 1000, 10000, 100000],
    'TauTau': [100, 1000, 10000, 100000],
    'BB': [100, 1000, 10000, 100000]
}

if category == 0:
    N_RECHIT_CUT_1 = 100
    N_RECHIT_CUT_2 = 100
elif category == 1:
    N_RECHIT_CUT_1 = 80
    N_RECHIT_CUT_2 = 80
else:
    N_RECHIT_CUT_1 = 80
    N_RECHIT_CUT_2 = 100
    
for t in tables:   
#     if not 'dd' in t:continue

    print('\\begin{table}[!h]')
    print('\\begin{center}')
    if '0p4' in t: print('\\caption{Signal yield in the signal region in binA, for 0.4 GeV LLP mass and various decay modes, assuming BR(H$\\rightarrow$ss) = 1\% .}')
    elif '1p0' in t: print('\\caption{Signal yield in the signal region in binA, for 1 GeV LLP mass and various decay modes, assuming BR(H$\\rightarrow$ss) = 1\% .}')
    elif '1p5' in t: print('\\caption{Signal yield in the signal region in binA, for 1.5 GeV LLP mass and various decay modes, assuming BR(H$\\rightarrow$ss) = 1\% .}')
    elif 'BB' in t: print('\\caption{Signal yield in the signal region in binA, for LLP decaying to 2 b quarks and various LLP masses, assuming BR(H$\\rightarrow$ss) = 1\% .}')
    elif 'dd' in t: print('\\caption{Signal yield in the signal region in binA, for LLP decaying to 2 d quarks and various LLP masses, assuming BR(H$\\rightarrow$ss) = 1\% .}')
    elif 'Tau' in t: print('\\caption{Signal yield in the signal region in binA, for LLP decaying to 2 $\\tau$ leptons and various LLP masses, assuming BR(H$\\rightarrow$ss) = 1\% .}')
    print('\\label{tab:signal_yield'+t+'}')
    print('\\begin{tabular}{|'+'l|'*(len(ctau_list[t])+1)+'}')


    
    
    for category in [0,1,2]:
        if category == 0:print("\\multicolumn{"+str(len(ctau_list[t])+1)+"}{c}{CSC-CSC}\\\\")
        elif category == 1:print("\\multicolumn{"+str(len(ctau_list[t])+1)+"}{c}{DT-DT}\\\\")
        else: print("\\multicolumn{"+str(len(ctau_list[t])+1)+"}{c}{DT-CSC}\\\\")
        print("\\hline")
        if 'dd' in t or 'BB' in t or 'Tau' in t:
            print('LLP mass / $c\\tau$ &', ' mm & '.join(map(str, [round(num,1) for num in ctau_list[t]])).replace('0.0','/'), 'mm \\\\')
        else:
            print('LLP decay / $c\\tau$ &', ' mm & '.join(map(str, [round(num,1) for num in ctau_list[t]])).replace('0.0','/'), 'mm \\\\')

        print("\\hline")


        for k in old_ctau.keys():
            if t not in k:continue
            m = k[k.find('ms')+2:].replace('p','.')
            if float(m) == int(float(m)):m = str(int(float(m)))
            signal_rate = []
            for ct in ctau_list[t]:
                signal = 0
                for p in prod:
                    key = 'MC_'+p+'_'+k + '_'+str(ct)+'_'+str(category)
                    if key not in sel_ev.keys():
                        signal += 1000000
                        continue
                    else:
                        if np.count_nonzero(sel_ev[key]) == 0: continue
                        # signal region
                        cond_d = np.logical_and(nhits1[key]>=N_RECHIT_CUT_1, nhits2[key]>=N_RECHIT_CUT_2)#bin A
                    signal += np.sum(weight[key][cond_d])
                signal_rate.append(signal)
            signal_rate = np.array(signal_rate)
    #         print(m, old_ctau[k])
    #         print(m, '\t', '\t'.join(map(str, [round(num,5) for num in signal_rate*BR])))
            if 'dd' in t or 'BB' in t or 'Tau' in t:
                print(m, 'GeV &', ' & '.join(map(str, [round(num,1) for num in signal_rate*BR])).replace('60000.0','/'), '\\\\')
            else:
                print(k[k.find('STo')+3:k.find('_ms')], ' &', ' & '.join(map(str, [round(num,1) for num in signal_rate*BR])).replace('60000.0','/'), '\\\\')
          
        print("\\hline")

    print('\\end{tabular}')
    print('\\end{center}')
    print('\\end{table}')
    print('\n')

\begin{table}[!h]
\begin{center}
\caption{Signal yield in the signal region in binA, for 0.4 GeV LLP mass and various decay modes, assuming BR(H$\rightarrow$ss) = 1\% .}
\label{tab:signal_yield0p4}
\begin{tabular}{|l|l|l|}
\multicolumn{3}{c}{CSC-CSC}\\
\hline
LLP decay / $c\tau$ & 10 mm & 50 mm \\
\hline
\hline
\multicolumn{3}{c}{DT-DT}\\
\hline
LLP decay / $c\tau$ & 10 mm & 50 mm \\
\hline
\hline
\multicolumn{3}{c}{DT-CSC}\\
\hline
LLP decay / $c\tau$ & 10 mm & 50 mm \\
\hline
\hline
\end{tabular}
\end{center}
\end{table}


\begin{table}[!h]
\begin{center}
\caption{Signal yield in the signal region in binA, for 1 GeV LLP mass and various decay modes, assuming BR(H$\rightarrow$ss) = 1\% .}
\label{tab:signal_yield1p0}
\begin{tabular}{|l|l|l|}
\multicolumn{3}{c}{CSC-CSC}\\
\hline
LLP decay / $c\tau$ & 25 mm & 125 mm \\
\hline
\hline
\multicolumn{3}{c}{DT-DT}\\
\hline
LLP decay / $c\tau$ & 25 mm & 125 mm \\
\hline
\hline
\multicolumn{3}{c}{DT-CSC}\\
\hline
LLP decay / $c\tau$ & 25 mm & 12

In [ ]:
\begin{table}[!h]
\begin{center}
\caption{Signal yield in the signal region in binA, for LLP decaying to 2 b quarks and various LLP masses, assuming BR(H$\rightarrow$ss) = 1\% .}
\label{tab:signal_yieldBB}
\begin{tabular}{|l|l|l|l|l|}
\multicolumn{5}{c}{CSC-CSC}\\
\hline
LLP mass / $c\tau$ & 100 mm & 1000 mm & 10000 mm & 100000 mm \\
\hline
15 GeV & 0.5 & 2.7 & 0.1 & 0.0 \\
40 GeV & 0.0 & 4.5 & 0.6 & 0.0 \\
55 GeV & 0.0 & 0.3 & 0.1 & 0.0 \\
\hline
\multicolumn{5}{c}{DT-DT}\\
\hline
LLP mass / $c\tau$ & 100 mm & 1000 mm & 10000 mm & 100000 mm \\
\hline
15 GeV & 0.1 & 7.5 & 0.2 & 0.0 \\
40 GeV & 0.0 & 7.0 & 2.2 & 0.0 \\
55 GeV & 0.0 & 1.1 & 0.8 & 0.0 \\
\hline
\multicolumn{5}{c}{DT-CSC}\\
\hline
LLP mass / $c\tau$ & 100 mm & 1000 mm & 10000 mm & 100000 mm \\
\hline
15 GeV & 0.2 & 6.5 & 0.2 & 0.0 \\
40 GeV & 0.0 & 5.6 & 1.6 & 0.0 \\
55 GeV & 0.0 & 1.3 & 1.2 & 0.1 \\
\hline
\end{tabular}
\end{center}
\end{table}